In [2]:
import json
import random
import time

from faker import Faker
from confluent_kafka import Producer
from datetime import datetime

# Kafka library yang dipakai: kafka-python
# Kalo udah terlanjur install kafka library, uninstall librarynya (pip uninstall kafka)

# Flink library yang dipakai: apache-flink

In [3]:
fake = Faker()

def generate_transactions():
  # user = fake.simple_profile()

  return {
    "transaction_id": fake.uuid4(),
    "customer_id": random.randint(1000,10000),
    'transaction_timestamp': datetime.now().strftime('%Y-%m-%dT%H:%M:%S.%f%z'),
    "merchant": random.choice(["KFC","McDonalds","Cek Min","Sour Sally", "Warung Zaonk", "Soto Semut","Mamank"]),
    "transaction_method": random.choice(["BRI","BCA","BNI","CIMB Niaga","Mandiri","BTN"]),
    "transaction_amt": round(random.uniform(10000, 1000000),2)
  }
def delivery_report(err, msg):
    if err is not None:
        print(f'Message delivery failed: {err}')
    else:
        print(f"Message delivered to {msg.topic} [{msg.partition()}]")

In [18]:
import socket
from kafka.admin import KafkaAdminClient, NewTopic

admin_client = KafkaAdminClient(
    bootstrap_servers="192.168.60.60:9092"
)

topic_list = []
topic_list.append(NewTopic(name="merchant_transactions10", num_partitions=1, replication_factor=1))
admin_client.create_topics(new_topics=topic_list, validate_only=False)

TopicAlreadyExistsError: [Error 36] TopicAlreadyExistsError: Request 'CreateTopicsRequest_v3(create_topic_requests=[(topic='merchant_transactions10', num_partitions=1, replication_factor=1, replica_assignment=[], configs=[])], timeout=30000, validate_only=False)' failed with response 'CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='merchant_transactions10', error_code=36, error_message="Topic 'merchant_transactions10' already exists.")])'.

In [4]:
from kafka import KafkaProducer
import json


topic_name = "merchant_transactions"

producer = KafkaProducer(bootstrap_servers="192.168.60.60:9092")

try:
  for i in range(1,5):
    transaction = generate_transactions()
    print(transaction)
    producer.send(
      topic=topic_name,
      key=transaction['transaction_id'].encode('utf-8'),
      value=json.dumps(transaction).encode('utf-8')
    )
    time.sleep(0.5)
  producer.flush()
  
except BufferError:
  print("Buffer full! Waiting...")
  time.sleep(1)
except Exception as E:
  print(E)

{'transaction_id': '408b6760-0e7b-4bb2-af6a-01d5a3f7d642', 'customer_id': 9358, 'transaction_timestamp': '2024-07-15T19:54:49.884445', 'merchant': 'Mamank', 'transaction_method': 'CIMB Niaga', 'transaction_amt': 182506.32}
{'transaction_id': '7f055eb8-f18c-47ee-b966-64e59e4cad8d', 'customer_id': 3557, 'transaction_timestamp': '2024-07-15T19:54:50.390069', 'merchant': 'McDonalds', 'transaction_method': 'BRI', 'transaction_amt': 956603.73}
{'transaction_id': '7d04b9ff-8853-46f0-bf56-d4a89c030088', 'customer_id': 5501, 'transaction_timestamp': '2024-07-15T19:54:50.891724', 'merchant': 'Cek Min', 'transaction_method': 'BRI', 'transaction_amt': 906710.05}
{'transaction_id': 'd700245b-bc9d-4c49-b780-88f5b18f8a00', 'customer_id': 8568, 'transaction_timestamp': '2024-07-15T19:54:51.392753', 'merchant': 'KFC', 'transaction_method': 'CIMB Niaga', 'transaction_amt': 504499.29}


In [43]:
import logging
import os 
import sys

from pyflink.datastream import StreamExecutionEnvironment, RuntimeExecutionMode
from pyflink.datastream.connectors.file_system import FileSource, StreamFormat, FileSink, OutputFileConfig, RollingPolicy
from pyflink.datastream.connectors.kafka import FlinkKafkaConsumer
from pyflink.common.serialization import SimpleStringSchema

import urllib

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

try:
  env = StreamExecutionEnvironment.get_execution_environment()

  # kafka_connector  = os.path.join(os.getcwd(), 'flink_jars','flink-sql-connector-kafka-3.2.0-1.19.jar')
  # kafka_client = os.path.join(os.getcwd(), 'flink_jars','kafka-clients-3.7.1.jar')
  kafka_connector  = os.path.abspath(os.path.join(os.getcwd(), 'flink_jars','flink-sql-connector-kafka-3.2.0-1.19.jar'))
  kafka_client = os.path.abspath(os.path.join(os.getcwd(), 'flink_jars','flink-sql-connector-kafka-3.2.0-1.19.jar'))


  env.add_jars(
    str("file:///"+ urllib.parse.quote(kafka_connector)),
    str("file:///"+ urllib.parse.quote(kafka_client))
  )

  topic_name = "merchant_transactions"
  kafka_consumer = FlinkKafkaConsumer(
      topics=topic_name,
      properties={'bootstrap.servers': 'http://192.168.60.60:9092',
                  'group.id': 'flink-group'},
      deserialization_schema= SimpleStringSchema(),
  )

  stream = env.add_source(kafka_consumer)
  stream.print()
  # stream.map(lambda value: f"Received: {value}").print()

  # # Execute the Flink job
  env.execute("Flink Kafka Consumer Example")
except Exception as E:
  logger.error(f"Error in Flink Kafka Consumer Job: {E}")

'file%3A//c%3A%5CUsers%5CSharing%20Vision%5CDocuments%5Cbackup%20hans%202%5CDE%5CFlink%5Cvagrant_folder%5Cflink_jars%5Cflink-sql-connector-kafka-3.2.0-1.19.jar'

Note: you may need to restart the kernel to use updated packages.


In [10]:
ds

In [6]:
kafka_connector

'c:\\Users\\Sharing Vision\\Documents\\backup hans 2\\DE\\Flink\\vagrant_folder\\flink_jars\\flink-sql-connector-kafka-3.2.0-1.19.jar'

In [9]:
from confluent_kafka.admin import AdminClient, NewTopic
import socket
test = socket.gethostname()
test

admin_client = AdminClient({
    "bootstrap.servers": "192.168.60.60:9092"
})

topic_list = []
topic_list.append(NewTopic("merchant_transaction", 1, 1))
fs = admin_client.create_topics(topic_list)

# Wait for each operation to finish
for topic, f in fs.items():
    try:
        f.result()  # The result itself is None
        print(f"Topic '{topic}' created successfully")
    except Exception as e:
        print(f"Failed to create topic '{topic}': {e}")

Topic 'merchant_transaction' created successfully


{'transaction_id': 'dae6ed07-20d7-44b0-b315-072c6f270468', 'customer_id': 9397, 'transaction_timestamp': '2024-07-10T09:55:18.877769', 'merchant': 'Cek Min', 'transaction_method': 'BRI', 'transaction_amt': 185731.41}
{'transaction_id': 'ab07b8a9-4fe9-4185-9db9-16708a4a1f04', 'customer_id': 8231, 'transaction_timestamp': '2024-07-10T09:55:19.384689', 'merchant': 'McDonalds', 'transaction_method': 'BCA', 'transaction_amt': 427372.12}
{'transaction_id': '63512fcf-de76-46bf-97b3-dcd9cb316dbb', 'customer_id': 4059, 'transaction_timestamp': '2024-07-10T09:55:19.885803', 'merchant': 'Sour Sally', 'transaction_method': 'BCA', 'transaction_amt': 78758.74}
{'transaction_id': 'f334cc96-702e-4af8-a1c5-4c2eb521b4a8', 'customer_id': 7945, 'transaction_timestamp': '2024-07-10T09:55:20.387421', 'merchant': 'McDonalds', 'transaction_method': 'BTN', 'transaction_amt': 763617.45}
{'transaction_id': '960b549f-d17c-452e-aa41-fe8c502d02d9', 'customer_id': 5913, 'transaction_timestamp': '2024-07-10T09:55:20.

In [13]:
producer.close()

In [7]:
transaction = generate_transactions()
print(json.dumps(transaction).encode('utf-8'))

b'{"transaction_id": "3a6cbb88-7981-4f40-be50-71e0439c5498", "customer_id": 1474, "transaction_timestamp": "2024-07-10T09:29:45.447735", "merchant": "Sour Sally", "transaction_method": "BNI", "transaction_amt": 915510.29}'
